Install openai-agents SDK
```



In [ ]:
!pip install -Uq openai-agents

Make your Jupyter Notebook capable of running asynchronous functions.


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:

!pip install --upgrade openai python-dotenv


import os
from dotenv import load_dotenv
from agents import Agent, Runner, AsyncOpenAI, set_default_openai_client, set_tracing_disabled, set_default_openai_api

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


set_tracing_disabled(True)
set_default_openai_api("chat_completions")
external_client = AsyncOpenAI(api_key="openai_api_key")
set_default_openai_client(external_client)


web_agent = Agent(
    name="WebDevAgent",
    instructions="You are a senior web developer. Solve only web development-related tasks.",
    model="gpt-4"
)

mobile_agent = Agent(
    name="MobileDevAgent",
    instructions="You are an expert in mobile app development. Focus only on mobile-related tasks.",
    model="gpt-4"
)

devops_agent = Agent(
    name="DevOpsAgent",
    instructions="You are a DevOps engineer. Handle infrastructure, CI/CD, and cloud tasks.",
    model="gpt-4"
)

panacloud_agent = Agent(
    name="PanacloudAgent",
    instructions="""
    You are a task router. Based on the user's request, decide which agent should handle it:
    - If it's web development → WebDevAgent
    - If it's mobile app related → MobileDevAgent
    - If it's infrastructure/devops → DevOpsAgent
    Reply with only one of the following: WebDevAgent, MobileDevAgent, or DevOpsAgent.
    """,
    model="gpt-4"
)


AGENT_MAP = {
    "webdevagent": web_agent,
    "mobiledevagent": mobile_agent,
    "devopsagent": devops_agent
}


def log_task(task, chosen_agent, response):
    with open("agent_log.txt", "a") as f:
        f.write(f" User Task: {task}\n")
        f.write(f" Chosen Agent: {chosen_agent}\n")
        f.write(f" Agent Response:\n{response}\n")
        f.write("="*60 + "\n")


def route_and_execute(user_task):
    print("🧠 Asking Panacloud to choose the right expert agent...")
    chosen = Runner.run_sync(panacloud_agent, user_task).final_output.strip().lower()
    print(f" Panacloud selected: {chosen}\n")

    selected_agent = AGENT_MAP.get(chosen)
    if selected_agent:
        response = Runner.run_sync(selected_agent, user_task).final_output
        log_task(user_task, chosen, response)
        return response
    else:
        return " Error: Panacloud could not determine a valid agent."

# ✅ Run it
task = input(" Enter your task for Agentic AI: ")
result = route_and_execute(task)

print("\n Final Agent Response:\n")
print(result)


💬 Enter your task for Agentic AI: python oop
🧠 Asking Panacloud to choose the right expert agent...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: openai_a**_key. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}